In [1]:
from models.model import ResNet50, ViTBase, CLIPModel
from models.utils import train_step, eval_step, DataLoaders
from models.transforms import transforms_resnet,transforms_clip_vit, transforms_vit
import torch
import torch.nn as nn
from torchvision.datasets import CIFAR10, CIFAR100, SVHN
from tqdm import tqdm
import pandas as pd
import warnings
warnings.filterwarnings("ignore")
BATCH_SIZE = 64
N_EPOCHS = 3

In [2]:
def train_and_eval(train_loader,test_loader,model,loss_fn,optimizer,device,modeltype):
    tr_metric = {"Accuracy":[],"Loss":[]}
    ts_metric = {"Accuracy":[],"Loss":[]}

    for epoch in tqdm(range(N_EPOCHS)):
        tr_loss, tr_acc = train_step(model,train_loader,loss_fn,optimizer,device,modeltype,data='svhn')
        ts_loss, ts_acc = eval_step(model,test_loader,loss_fn,device,modeltype,data="svhn")

        tr_metric["Accuracy"].append(tr_acc)
        tr_metric["Loss"].append(tr_loss)

        ts_metric["Accuracy"].append(ts_acc)
        ts_metric["Loss"].append(ts_loss)
    
    return tr_metric, ts_metric

In [2]:
train = SVHN(root='data',transform=transforms_resnet,download=True)
test = SVHN(root='data',transform=transforms_resnet,split="test",download=True)

dl = DataLoaders(train,test,'resnet',BATCH_SIZE,True,'svhn')
train_loader, test_loader = dl.get_loaders()

resnet_model = ResNet50(n_classes=10)
loss_func = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(resnet_model.parameters())
device = 'cuda' if torch.cuda.is_available() else 'cpu'


tr,ts = train_and_eval(train_loader,test_loader,resnet_model,loss_func,optimizer,device,'resnet50')

print("Final Train Accuracy:",tr["Accuracy"][-1])
print("Final Test Accuracy:",ts["Accuracy"][-1])
print("Final Train Loss:",tr["Loss"][-1])
print("Final Test Loss:",ts["Loss"][-1])

pd.DataFrame(tr).to_csv("svhn_resnet50_tr.csv") 
pd.DataFrame(ts).to_csv("svhn_resnet50_ts.csv")

Using downloaded and verified file: data\train_32x32.mat
Using downloaded and verified file: data\test_32x32.mat


100%|██████████| 3/3 [06:29<00:00, 129.69s/it]

Final Train Accuracy: 0.49746782
Final Test Accuracy: 0.5081822
Final Train Loss: 1.4906173232340916
Final Test Loss: 1.4485609182854542


In [3]:
train = SVHN(root='data',transform=transforms_vit,download=True)
test = SVHN(root='data',transform=transforms_vit,split="test",download=True)
dl = DataLoaders(train,test,'vit',BATCH_SIZE,True,'svhn')
train_loader, test_loader = dl.get_loaders()

vit_model = ViTBase(n_classes=10)
loss_func = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(vit_model.parameters())
device = 'cuda' if torch.cuda.is_available() else 'cpu'


tr,ts = train_and_eval(train_loader,test_loader,vit_model,loss_func,optimizer,device,'vit')

print("Final Train Accuracy:",tr["Accuracy"][-1])
print("Final Test Accuracy:",ts["Accuracy"][-1])
print("Final Train Loss:",tr["Loss"][-1])
print("Final Test Loss:",ts["Loss"][-1])


pd.DataFrame(tr).to_csv("svhn_vitbase_tr.csv") 
pd.DataFrame(ts).to_csv("svhn_vitbase_ts.csv")

Using downloaded and verified file: data\train_32x32.mat
Using downloaded and verified file: data\test_32x32.mat


  0%|          | 0/3 [00:31<?, ?it/s]


KeyboardInterrupt: 

In [3]:
train = SVHN(root='data',download=True)
test = SVHN(root='data',split="test",download=True)
dl = DataLoaders(train,test,'clip',BATCH_SIZE,True,'svhn')
train_loader, test_loader = dl.get_loaders()

clip_model = CLIPModel(256)
loss_func = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(clip_model.parameters())
device = 'cuda' if torch.cuda.is_available() else 'cpu'


tr,ts = train_and_eval(train_loader,test_loader,clip_model,loss_func,optimizer,device,'clip')

print("Final Train Accuracy:",tr["Accuracy"][-1])
print("Final Test Accuracy:",ts["Accuracy"][-1])
print("Final Train Loss:",tr["Loss"][-1])
print("Final Test Loss:",ts["Loss"][-1])


pd.DataFrame(tr).to_csv("svhn_clip_tr.csv") 
pd.DataFrame(ts).to_csv("svhn_clip_ts.csv")

Using downloaded and verified file: data\train_32x32.mat
Using downloaded and verified file: data\test_32x32.mat


100%|██████████| 3/3 [06:16<00:00, 125.64s/it]

Final Train Accuracy: 0.4115046
Final Test Accuracy: 0.5625384
Final Train Loss: 7.575412401571681
Final Test Loss: 2.2214770694915433
